In [49]:
import pandas as pd

In [50]:
transaction_all = pd.read_csv('../../데이터/transaction/transaction_all.csv')

In [51]:
transaction_all['계약연도'] = transaction_all['계약년월'].apply(lambda x: str(x)[:4])

transaction_all['평수'] = transaction_all['전용면적(㎡)'].apply(lambda x : x // 3.3)
transaction_all['평수대'] = (transaction_all['평수'] // 10 * 10).astype(str) + '~' + ((transaction_all['평수'] // 10 + 1) * 10).astype(str) + '평'

transaction_all['거래금액(만원)'] = transaction_all['거래금액(만원)'].apply(lambda x: int(x.replace(',','')))
transaction_all['평단가'] = transaction_all['거래금액(만원)'] / transaction_all['평수']

### 거래량 높은 평수 찾고 해당 평수만 남기기

In [52]:
transaction_all.groupby(['단지명', '계약연도', '평수대'])['평단가'].mean().reset_index()

,단지명,계약연도,평수대,평단가
0,(1-10),2014,20.0~30.0평,681.818182
1,(1-102),2007,0.0~10.0평,875.000000
2,(1-102),2007,10.0~20.0평,809.259259
3,(1-102),2007,20.0~30.0평,571.428571
4,(1-102),2014,10.0~20.0평,944.444444
...,...,...,...,...
130768,힐하우스(802-3),2009,60.0~70.0평,1482.587065
130769,힐하우스(802-3),2010,50.0~60.0평,1423.728814
130770,힐하우스(802-3),2010,60.0~70.0평,1406.250000
130771,힐하우스(802-3),2012,60.0~70.0평,1194.029851


In [53]:
transaction_all['평수대'].value_counts()

20.0~30.0평      522439
10.0~20.0평      484126
30.0~40.0평      135790
40.0~50.0평       43205
0.0~10.0평        36311
50.0~60.0평        9758
60.0~70.0평        3457
70.0~80.0평        2360
80.0~90.0평         111
90.0~100.0평          8
120.0~130.0평         2
110.0~120.0평         1
Name: 평수대, dtype: int64

In [54]:
transaction_all.groupby(['계약연도', '평수대']).size().unstack().fillna(0)

평수대,0.0~10.0평,10.0~20.0평,110.0~120.0평,120.0~130.0평,20.0~30.0평,30.0~40.0평,40.0~50.0평,50.0~60.0평,60.0~70.0평,70.0~80.0평,80.0~90.0평,90.0~100.0평
계약연도,,,,,,,,,,,,
2006,1220.0,51724.0,0.0,1.0,51269.0,12314.0,3303.0,600.0,198.0,178.0,6.0,0.0
2007,1138.0,31265.0,0.0,0.0,21481.0,3721.0,1020.0,222.0,85.0,75.0,4.0,0.0
2008,905.0,24802.0,0.0,0.0,25080.0,4810.0,1305.0,231.0,112.0,58.0,4.0,0.0
2009,1174.0,29320.0,0.0,0.0,31090.0,8247.0,2872.0,599.0,176.0,175.0,11.0,1.0
2010,733.0,17652.0,0.0,0.0,18547.0,4926.0,1984.0,447.0,129.0,93.0,4.0,0.0
2011,808.0,22886.0,0.0,0.0,22945.0,5595.0,1772.0,421.0,130.0,82.0,2.0,0.0
2012,1324.0,16232.0,0.0,0.0,17112.0,4381.0,1448.0,344.0,153.0,81.0,4.0,0.0
2013,1739.0,26104.0,0.0,0.0,30032.0,7214.0,2303.0,512.0,193.0,108.0,4.0,1.0
2014,2733.0,31853.0,0.0,0.0,36885.0,10033.0,2979.0,755.0,254.0,151.0,3.0,0.0


In [55]:
# 20~30평 
transaction_all = transaction_all[transaction_all['평수대']=='20.0~30.0평']

### 건축 이후 36개월 이상 연속으로 거래 없는 단지 제거

In [56]:
def find_reconstruction_complex(constructed_year, dates):
    if (pd.to_datetime('202212', format='%Y%m').year - pd.to_datetime(dates[-1], format='%Y%m').year) * 12 + pd.to_datetime('202212', format='%Y%m').month - pd.to_datetime(dates[-1], format='%Y%m').month > 36:
        return True
    else:
        return False

transaction_all.dropna(subset=['건축년도'], inplace=True)
constructed_years = transaction_all.groupby('단지명')['건축년도'].first().astype(int).to_dict()
contract_dates = transaction_all.groupby('단지명')['계약년월'].unique().to_dict()
reconstructed_complexes = [complex_name for complex_name, dates in contract_dates.items() if find_reconstruction_complex(constructed_years[complex_name], dates)]

print(len(transaction_all['단지명'].unique()))
transaction_all = transaction_all[~transaction_all['단지명'].isin(reconstructed_complexes)].reset_index(drop=True)
print(len(transaction_all['단지명'].unique()))

5169
4308


### 단지별로 거래가 없는 달 선형보간 처리

In [57]:
transaction_all = transaction_all[['시군구','단지명','계약년월','평단가']]
transaction_all['계약년월'] = pd.to_datetime(transaction_all['계약년월'], format='%Y%m')
transaction_all.drop_duplicates(subset=['시군구','단지명','계약년월'], keep='first', inplace=True)
transaction_all.reset_index(drop=True, inplace=True)

In [58]:
grouped_transaction_all = pd.DataFrame(transaction_all.groupby(['시군구','단지명']).apply(lambda group: group.set_index('계약년월')['평단가']))
grouped_transaction_all

평단가
시군구           단지명            계약년월                   
서울특별시 강남구 개포동 개포2차현대아파트(220) 2006-03-01  2586.956522
                             2006-04-01  2913.043478
                             2006-06-01  2608.695652
                             2006-10-01  3141.304348
                             2006-12-01  3326.086957
...                                              ...
서울특별시 중랑구 중화동 현대휴앤미(102동)    2015-05-01  1195.652174
                             2015-06-01  1247.500000
                             2015-10-01  1228.260870
                             2020-09-01  1739.130435
                             2022-04-01  2304.347826

[246685 rows x 1 columns]

In [59]:
def price_per_pyeong_interpolate(group):
    idx = pd.date_range(group['계약년월'].min(), group['계약년월'].max(), freq='MS')
    group = group.set_index('계약년월').reindex(idx)
    group['단지명'] = group['단지명'].fillna(method='ffill')
    group['시군구'] = group['시군구'].fillna(method='ffill')
    group['평단가'] = group['평단가'].interpolate()
    return group

interpolated_transaction_all = pd.DataFrame(transaction_all.groupby(['시군구','단지명']).apply(price_per_pyeong_interpolate)['평단가']).reset_index().rename(columns={'level_2': '계약년월'})
interpolated_transaction_all

,시군구,단지명,계약년월,평단가
0,서울특별시 강남구 개포동,개포2차현대아파트(220),2006-03-01,2586.956522
1,서울특별시 강남구 개포동,개포2차현대아파트(220),2006-04-01,2913.043478
2,서울특별시 강남구 개포동,개포2차현대아파트(220),2006-05-01,2760.869565
3,서울특별시 강남구 개포동,개포2차현대아파트(220),2006-06-01,2608.695652
4,서울특별시 강남구 개포동,개포2차현대아파트(220),2006-07-01,2741.847826
...,...,...,...,...
853794,서울특별시 중랑구 중화동,현대휴앤미(102동),2021-12-01,2185.354691
853795,서울특별시 중랑구 중화동,현대휴앤미(102동),2022-01-01,2215.102975
853796,서울특별시 중랑구 중화동,현대휴앤미(102동),2022-02-01,2244.851259
853797,서울특별시 중랑구 중화동,현대휴앤미(102동),2022-03-01,2274.599542


In [60]:
grouped_interpolated_transaction_all = pd.DataFrame(interpolated_transaction_all.groupby(['시군구','단지명']).apply(lambda group: group.set_index('계약년월')['평단가']))
grouped_interpolated_transaction_all

평단가
시군구           단지명            계약년월                   
서울특별시 강남구 개포동 개포2차현대아파트(220) 2006-03-01  2586.956522
                             2006-04-01  2913.043478
                             2006-05-01  2760.869565
                             2006-06-01  2608.695652
                             2006-07-01  2741.847826
...                                              ...
서울특별시 중랑구 중화동 현대휴앤미(102동)    2021-12-01  2185.354691
                             2022-01-01  2215.102975
                             2022-02-01  2244.851259
                             2022-03-01  2274.599542
                             2022-04-01  2304.347826

[853799 rows x 1 columns]

### 단지별로 계약년월 200601~202212에서 없는 평단가는 0으로 채우기

In [61]:
months = pd.to_datetime(pd.date_range(start="2006-01-01", end="2022-12-01", freq='MS').strftime('%Y-%m-%01'))
complexes = interpolated_transaction_all['단지명'].unique()

combinations = pd.DataFrame({
    '단지명': pd.Series(complexes).repeat(len(months)),
    '계약년월': months.tolist() * len(complexes)
})

filled_interpolated_transaction_all = pd.merge(combinations, interpolated_transaction_all, on=['단지명', '계약년월'], how='left')
filled_interpolated_transaction_all['평단가'].fillna(0, inplace=True)
filled_interpolated_transaction_all['시군구'] = filled_interpolated_transaction_all.groupby('단지명')['시군구'].transform('first')

In [63]:
filled_interpolated_transaction_all

,단지명,계약년월,시군구,평단가
0,개포2차현대아파트(220),2006-01-01,서울특별시 강남구 개포동,0.000000
1,개포2차현대아파트(220),2006-02-01,서울특별시 강남구 개포동,0.000000
2,개포2차현대아파트(220),2006-03-01,서울특별시 강남구 개포동,2586.956522
3,개포2차현대아파트(220),2006-04-01,서울특별시 강남구 개포동,2913.043478
4,개포2차현대아파트(220),2006-05-01,서울특별시 강남구 개포동,2760.869565
...,...,...,...,...
1028066,현대휴앤미(102동),2022-08-01,서울특별시 중랑구 중화동,0.000000
1028067,현대휴앤미(102동),2022-09-01,서울특별시 중랑구 중화동,0.000000
1028068,현대휴앤미(102동),2022-10-01,서울특별시 중랑구 중화동,0.000000
1028069,현대휴앤미(102동),2022-11-01,서울특별시 중랑구 중화동,0.000000
